In [ ]:
# cnn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

# Hyper parameters
EPOCHS = 40
BATCH_SIZE = 64

# dataset load
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307, ), (0.3081, ))
                          ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307, ), (0.3081, ))
                          ])),
    batch_size=BATCH_SIZE, shuffle=True)


# Convolution Net
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        self.drop = nn.Dropout2d()

        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))

        x = x.view(-1, 320)

        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if(batch_idx % 200 == 0):
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.
                  format(epoch, batch_idx * len(data),
                         len(train_loader.dataset),
                         100. * batch_idx / len(train_loader),
                         loss.item()))


def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차 합산
            test_loss += F.cross_entropy(output, target,
                                    reduction='sum').item()

            # 가장 큰 값의 인덱스가 예측값임
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy



# run
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print("[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%".format(
        epoch, test_loss, test_accuracy
    ))

In [ ]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if(batch_idx % 200 == 0):
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.
                  format(epoch, batch_idx * len(data),
                         len(train_loader.dataset),
                         100. * batch_idx / len(train_loader),
                         loss.item()))


def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차 합산
            test_loss += F.cross_entropy(output, target,
                                    reduction='sum').item()

            # 가장 큰 값의 인덱스가 예측값임
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy





# ResNET, color dataset, residual network
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")



# color셋은 흑백보다 복잡하므로 학습을 더 많이해야함.
EPOCHS = 300
BATCH_SIZE = 128

# CIFAR-10 데이터셋 사용 32x32
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=True,
                     download=True,
                     transform=transforms.Compose([
                         transforms.RandomCrop(32, padding=4),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5),
                                             (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True )

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=False,
                     download=True,
                     transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5),
                                              (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True )


# in_planes: 입력채널수 , planes : 출력 채널수
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)


        # nn.Sequential()은 여러 모듈을 하나로 묶는 역할
        # 이 shortcut은 입력텐서와 출력텐서의 크기를 맞추기 위함.
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out




class ResNet(nn.Module):
    def __init__(self, num_classed=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classed)


    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50,
                                      gamma=0.1)

#print(model) # 모델의 모습 확인하기


for epoch in range(1, EPOCHS + 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(
        epoch, test_loss, test_accuracy ))



Files already downloaded and verified
Files already downloaded and verified
Train Epoch : 1 [0/50000 (0%)]	Loss: 2.374475
Train Epoch : 1 [25600/50000 (51%)]	Loss: 1.568350
[1] Test Loss : 1.3790, Accuracy : 50.24%
Train Epoch : 2 [0/50000 (0%)]	Loss: 1.288828
Train Epoch : 2 [25600/50000 (51%)]	Loss: 1.189685
[2] Test Loss : 1.2402, Accuracy : 56.44%
Train Epoch : 3 [0/50000 (0%)]	Loss: 0.984990
Train Epoch : 3 [25600/50000 (51%)]	Loss: 0.854809
[3] Test Loss : 1.0919, Accuracy : 63.89%
Train Epoch : 4 [0/50000 (0%)]	Loss: 0.896236
Train Epoch : 4 [25600/50000 (51%)]	Loss: 0.740307
[4] Test Loss : 1.1433, Accuracy : 62.22%
Train Epoch : 5 [0/50000 (0%)]	Loss: 0.688749
Train Epoch : 5 [25600/50000 (51%)]	Loss: 0.709998
[5] Test Loss : 1.0837, Accuracy : 66.46%
Train Epoch : 6 [0/50000 (0%)]	Loss: 0.754176
Train Epoch : 6 [25600/50000 (51%)]	Loss: 0.609063
[6] Test Loss : 0.7218, Accuracy : 76.18%
Train Epoch : 7 [0/50000 (0%)]	Loss: 0.793529
Train Epoch : 7 [25600/50000 (51%)]	Loss: 0.